In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

from multiresticodm.config import Config
from multiresticodm.outputs import Outputs
from multiresticodm.utils.misc_utils import *
from multiresticodm.utils.math_utils import *
from multiresticodm.utils.probability_utils import *

In [2]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

In [3]:
# Get important paths
experiment_dir = '/home/iz230/MultiResTICODM/data/outputs/cambridge_work_commuter_lsoas_to_msoas/exp1/JointTableSIM_MCMC_SweepedNoise_16_05_2023_20_09_04/'
config_path = os.path.join(experiment_dir,'config.json')

In [4]:
# Output processing settings
settings = {
    "logging_mode": "INFO",
    "coordinate_slice": ["da.loss_name.isin([str(['dest_attraction_ts_likelihood_loss']),str(['dest_attraction_ts_likelihood_loss', 'table_likelihood_loss'])])"],
    "metadata_keys":[],
    "burnin_thinning_trimming": [{'iter': {"burnin":10000, "thinning":10, "trimming":1000}}],
    "n_workers": 1,
    "filename_ending":"test",
    "sample":["table","intensity"],
    "force_reload":False
}

In [5]:
# Read config
conf = Config(
    path = config_path
)
conf.get_sweep_data()

56:23.721 config INFO ----------------------------------------------------------------------------------
56:23.734 config INFO /home/iz230/MultiResTICODM/data/outputs/
56:23.747 config INFO Parameter space size: 
 --- sigma: ['sigma', 'to_learn', 'covariance'] (2)
 --- title: ['title', 'axes', 'cells', 'name', 'proposal'] (6)
56:23.761 config INFO Total = 12.
56:23.774 config INFO ----------------------------------------------------------------------------------


In [6]:
outputs = Outputs(
    config = conf,
    settings = settings,
    base_dir = experiment_dir,
    console_handling_level = 'INFO',
    inputs = None,
    logger = conf.logger,
    print_slice = True
)

56:23.844 outputs INFO //////////////////////////////////////////////////////////////////////////////////
56:23.856 outputs INFO Slicing coordinates:
56:23.869 outputs INFO loss_name.isin([str(['dest_attraction_ts_likelihood_loss']),str(['dest_attraction_ts_likelihood_loss', 'table_likelihood_loss'])])
56:23.882 outputs INFO iter: burnin = 10000, thinning = 10, trimming = 1000
56:23.894 outputs INFO //////////////////////////////////////////////////////////////////////////////////


In [7]:
# Collect outputs from folder
outputs.load(indx = 0)

56:24.111 outputs INFO Reading samples alpha, beta, log_destination_attraction, sign, table.
Slicing coordinates sequentially:   0%|          | 0/60 [00:00<?, ?it/s]56:40.376 outputs SUCCESS Slicing ['iter'] {'burnin': 10000, 'thinning': 10, 'trimming': 1000} succeded {'iter': 1000}


In [8]:
data = []
for i in tqdm(range(len(outputs.data))):
    mean_intensity = outputs.get(i).compute_statistic(
        data = outputs.get(i).get_sample('intensity'),
        sample_name = 'intensity',
        statistic = 'signedmean',
        dim = ['id']
    )
    intensity_srmse = srmse(
        prediction = mean_intensity,
        ground_truth = outputs.get(0).get_sample('ground_truth_table')
    )
    

    datum = dict(zip(
        outputs.config.sweep_param_names,
        mean_intensity['sweep'].values[0]
    ))
    datum['intensity_srmse'] = intensity_srmse.values[0]
    try:
        mean_table = outputs.get(i).compute_statistic(
            data = outputs.get(i).get_sample('table'),
            sample_name = 'table',
            statistic = 'mean',
            dim = ['id']
        )
        table_srmse = srmse(
            prediction = mean_intensity,
            ground_truth = outputs.get(0).get_sample('ground_truth_table')
        )
        datum['table_srmse'] = table_srmse.values[0]
    except:
        pass
    data.append(datum)

  0%|          | 0/12 [00:00<?, ?it/s]

self.dim_names ['origin', 'destination']


AttributeError: 'TotallyConstrainedSIM' object has no attribute 'dim_names'

In [ ]:
pd.DataFrame.from_records(data)

In [ ]:
srmse(
    prediction = mean_intensity,
    ground_truth = outputs.get(0).get_sample('ground_truth_table')
)